In [2]:
""""Code to format info on movies in a csv file for publication."""

# built-in libraries
import csv
import datetime
from email.message import EmailMessage
import os.path
import re
import smtplib
import sqlite3
import sys

# installed with pip
import pandas as pd
#from sqlalchemy import create_engine

# Create list of dictionaries for each row.
listings = []
scrapedir = '/Users/Doug/workspace/cinema/scrapes/'
scrapefile = 'fandango-2019-06-13.csv'
scrape_path = scrapedir + scrapefile
with open(scrape_path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        layout_dct = {
            #'addy': row[0],
            'theater': row[1],
            'title': row[2],
            'mpaa': row[3],
            #'stars': row[4],
            'times': row[5],
            'format': row[6],
            #'synopsis': row[7],
            'date': row[8],
            'city': row[9]
        }
        listings.append(layout_dct)
        
del listings[0] #remove header row   

# Create Pandas dataframe for listings
df = pd.DataFrame(listings)
# Remove some duplicate listings
df = df[df.city != "Belleview, FL"]
df = df[df.city != "High Springs, FL"]
df = df[(df.city != "The Villages, FL") & (df.theater != "Belleview Cinemas")]
df = df[(df.city != "The Villages, FL") & (df.theater != "Old Mill Playhouse")]
df = df[(df.city != "The Villages, FL") & (df.theater != "Old Mill Playhouse")]
df.times = df.times.str.replace('p ', ' p.m.')
df.times = df.times.str.replace('a ', ' a.m.')
df.times = df.times.str.replace(':00', '')
df.times = df.times.str.replace('|', ', ')
#df.times = df.times.str.split('|', expand=True).rename(columns=lambda x: f"time_{x+1}"))
#df.times = df.times.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
#df.times = df.times.apply(lambda x: x.str.replace('p', ' p.m.') if x.dtype == "object" else x)
#df.times = df.times.apply(lambda x: x.str.replace('a', ' a.m.') if x.dtype == "object" else x)
#df.times = df.times.apply(lambda x: x.str.replace(':00', '') if x.dtype == "object" else x)
#df = df.drop('times', axis=1)
#df = pd.concat([df, times], axis=1, join_axes=[df.index])
#df['streetaddy'] = df.addy.str.split(',').str[0]
#df.streetaddy = df.streetaddy.str.replace('Blvd', 'Blvd.')
#df.streetaddy = df.streetaddy.str.replace('Blvd..', 'Blvd.')
#df.streetaddy = df.streetaddy.str.replace('Avenue', 'Ave')
#df.streetaddy = df.streetaddy.str.replace('Ave', 'Ave.')
#df.streetaddy = df.streetaddy.str.replace('Ave..', 'Ave.')
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['movieday'] = df['date'].dt.weekday_name
df = df.drop('city', axis=1)
#df['cityaddy'] = df.addy.str.split(',').str[1]
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
#df = df.drop('addy', axis=1)
#df = df.drop('stars', axis=1)
#df = df.drop('synopsis', axis=1)
df = df.drop('date', axis=1)
df['screen'] = df['title'].str.cat(df['format'],sep=", ")
df.screen = df.screen.str.cat(df['mpaa'],sep=", ")
df.screen = df.screen.str.cat(df['movieday'],sep=", ")
df = df.drop(['title', 'format', 'mpaa', 'movieday'],axis=1)

# Ocala market
oc6_df = df.loc[df['theater'] == 'Ocala Center 6']
ocd_dr = df.loc[df['theater'] == 'Ocala Drive-In']
bv_df = df.loc[df['theater'] == 'Belleview Cinemas']
mt_df = df.loc[df['theater'] == 'Marion Theatre']
omp_df = df.loc[df['theater'] == 'Old Mill Playhouse']
regaloca_df = df.loc[df['theater'] == 'Regal Hollywood Stadium 16 & IMAX - Ocala']
rialto_df = df.loc[df['theater'] == 'Rialto Theatre Spanish Springs Town Square']
amc_df = df.loc[df['theater'] == 'AMC Lake Square 12']
barn_df = df.loc[df['theater'] == 'Barnstorm Theater']

# Gainesville market
celeb_df = df.loc[df['theater'] == 'Celebration Pointe 10']
hipp_df = df.loc[df['theater'] == 'Hippodrome State Theatre']
butler_df = df.loc[df['theater'] == 'Regal Butler Town Center 14']
regalgnv_df = df.loc[df['theater'] == 'Regal Royal Park Stadium 16']
uac_df = df.loc[df['theater'] == 'UA Cinema 90 6']
starke_df = df.loc[df['theater'] == 'Florida Twin']

oc6_df

# need to group by movie title, format

,theater,times,screen
23,Ocala Center 6,"1 p.m., 4 p.m., 7 p.m., 9:45p","Men in Black: International, Standard, PG-13, ..."
38,Ocala Center 6,"1 p.m., 4 p.m., 7 p.m., 9:45p","Men in Black: International, Standard, PG-13, ..."
131,Ocala Center 6,"1 p.m., 4 p.m., 7 p.m., 9:45p","Godzilla: King of the Monsters (2019), Standar..."
142,Ocala Center 6,"1 p.m., 4 p.m., 7 p.m., 9:45p","Dark Phoenix, Standard, PG-13, Sunday"
152,Ocala Center 6,"1 p.m., 4 p.m., 7 p.m., 9:45p","Godzilla: King of the Monsters (2019), Standar..."
228,Ocala Center 6,"1 p.m., 4 p.m., 7 p.m., 9:45p","Dark Phoenix, Standard, PG-13, Friday"
236,Ocala Center 6,"1 p.m., 4 p.m., 7 p.m., 9:45p","The Secret Life of Pets 2, Standard, PG, Sunday"
247,Ocala Center 6,"1 p.m., 4 p.m., 7 p.m., 9:45p","Men in Black: International, Standard, PG-13, ..."
252,Ocala Center 6,"1 p.m., 4 p.m., 7 p.m., 9:45p","Dark Phoenix, Standard, PG-13, Saturday"
263,Ocala Center 6,"1 p.m., 4 p.m., 7 p.m., 9:45p","Aladdin (2019), Standard, PG, Sunday"


In [ ]:
def ocala6(oc6_df):
    
    for row in df.itertuples(index=False):
        if oc6_df.screen            
        

In [ ]:
df['screen'] = df['title'].str.cat(df['format'],sep=", ")
df.head()

In [ ]:
list(oc6_df.title.unique())
oc6_df["screen"] = oc6_df["title"].astype(str) + ", " + oc6_df["format"].astype(str)


In [ ]:
import csv
from datetime import datetime

scrapedir = '/Users/Doug/workspace/cinema/scrapes/'
scrapefile = 'fandango-2019-06-13.csv'
newfile = scrapedir + scrapefile

def ocadaily_pn(newfile):
    """ Builds print narrative for Ocala daily editions """
    global listings
    listing = []
    listings = []
    ocala_market = 'Ocala, FL'
    gnv_market = ['Gainesville, FL', 'Starke, FL']
    weekDays = ("Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday")
    with open(newfile) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        headers = next(csv_reader)
        for row in csv_reader:
            if row[9] in gnv_market:
                row[0] = row[0].split(', FL')[0]
                row[5] = row[5].replace('p | ', ' p.m., ')
                row[5] = row[5].replace('a | ', ' a.m., ')
                row[5] = row[5].replace('0p', '0 p.m.')
                row[5] = row[5].replace('5p', '5 p.m.')
                row[5] = row[5].replace(':00', '')
                row[8] = datetime.strptime(row[8], '%Y-%m-%d').weekday()
                movieday = weekDays[row[8]]
                movie_stuff = f"\"{row[2]}\", {row[3]}, {row[4]}, {row[6]}, " + movieday + f" at {row[5]}\n"
                theater_stuff = f'{row[1]}, {row[0]}, '
                listing = theater_stuff + movie_stuff
                listings.append(listing)
    
    return listings

if __name__ == '__main__':
    ocadaily_pn(newfile)
    print(listings)

In [ ]:
import csv
from datetime import datetime

scrapedir = '/Users/rayd/workspace/cinema/scrapes/'
scrapefile = 'fandango-2019-06-13.csv'
newfile = scrapedir + scrapefile

def ocadaily_pn(newfile):
    """ Builds print narrative for Ocala daily editions """
    global listings
    listing = []
    listings = []
    ocala_market = 'Ocala, FL'
    gnv_market = ['Gainesville, FL', 'Starke, FL']
    weekDays = ("Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday")
    with open(newfile) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        headers = next(csv_reader)
        for row in csv_reader:
            row[0] = row[0].split(', FL')[0]
            row[5] = row[5].replace('p | ', ' p.m., ')
            row[5] = row[5].replace('a | ', ' a.m., ')
            row[5] = row[5].replace('0p', '0 p.m.')
            row[5] = row[5].replace('5p', '5 p.m.')
            row[5] = row[5].replace(':00', '')
            row[8] = datetime.strptime(row[8], '%Y-%m-%d').weekday()
            movieday = weekDays[row[8]]
            movie_stuff = f"\"{row[2]}\", {row[3]}, {row[4]}, {row[6]}, " + movieday + f" at {row[5]}\n"
            theater_stuff = f'{row[1]}, {row[0]}, '
            listing = theater_stuff + movie_stuff
            listings.append(listing)
    
    return listings

if __name__ == '__main__':
    ocadaily_pn(newfile)
    print(listings)

In [ ]:
with open(txt_file, "w") as my_output_file:
    with open(csv_file, "r") as my_input_file:
        [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
    my_output_file.close()